###### Università degli Studi di Milano, Data Science and Economics Master Degree

# Multi-agent reinforcement learning

### Luigi Foscari

<small>see sections 1.1 to 1.4 and 5.2 to 5.4 of [Multi-Agent Reinforcement Learning: Foundations and Modern Approaches](https://www.marl-book.com) for a more in-depth overview. Consider watching this [YouTube video](https://www.youtube.com/watch?v=QfYx5q0Q75M) by the book's author which provides a great introduction.</small>

So far we tackled the problem of learning an optimal policy for a single agent inside a MDP, we now shift our focus towards contexts in which there are _multiple_ agents acting in the MDP with (possibly) different goals. For example, consider a fleet of autonomous vehicles with the goal of driving as quickly and safely as possible, a team of players in a competitive videogame or a group of automated traders that manage each their own investments in a public market.

**How does the addition of other learning agents change what we learned so far?**

_See the available [environments](https://pettingzoo.farama.org/environments/classic/) from the PettinZoo library for more examples_

A **multi-agent system** is composed of an environment and multiple decision-making agents, also called _players_, that interact in the environment to achieve certain goals. The environment evolves during play according the player's actions and optionally randomness. 

**Multi-agent reinforcement learning** or **MARL** tackles learning in such systems by devising algorithms that let the players refine their respective strategies, which can be collaborative, competitive or a mix of the two.

At the start of a round of the game, each player sees the state of the environment and picks an action, the round ends after every player has chosen an action, then the envoriment evolves accordingly and each player collects their reward to update their respective policy.

<img src="imgs/environment-marl.png" width="600" style="display: block; margin: 0 auto"/>

From the point of view of the agent, little has changed, they still have a set of actions, a policy, an enviroment and the goal of optimizing their policy.
What changed? The difference lies in the fact that **the immediate reward achieved by any player depends on the actions of the other players on the same round**.

For example consider a group of learning agents playing football, when a goal is scored, the success is tied to the actions of all the players, not only of the player that last kicked the ball. This introduces the problem of understanding how much of each player contributed, this is called **multi-agent credit assignment**.

Another issue that arises in MARL is the dependence on the number of agents, when considering games with a thousands of players, our learning procedure might struggle. When dealing with multiple agents, the algorithms will need to scale well with the number of agents.

### Central learning
One natural approach to the problem of learning with multiple agents is to use a single learning procedure (e.g. Q-learning) which learns the best strategy for each player, like a conductor in an orchestra, we shall call this algorithm the _maestro_. If each player $i \in \{1, \dots, N\}$ has an action space $\mathcal{A}_i$, then the maestro will have action space $\mathcal{A} = \mathcal{A}_1 \times \dots \times \mathcal{A}_N$, the state space is $\mathcal{S}$ and on each round the maestro chooses an action $a \in \mathcal{A}$ (where $a = (a_1, \dots, a_N)$ corresponds to actions for every single player) and distributes the correponding actions to the players, which they dutifully play as instructed. This approach can work, but it has a number of drawbacks:
- The actions space of the maestro can grows very quickly, if for example each player's action space has fixed size $A$, then the action space of the maestro has size $A^N$, which is exponential in the number of players. If each agent used an independent learning algorithm, each would have to manage only $A$ actions.
- In certain context centralized learning is not allowed, for example with autonomous driving or when the details of each agent are not known.
- Each player might have a different reward function (i.e. a different objective) and the maestro should take this into account in order to devise a complex learning objective suited to the underlying players. In practice the reward of each player might not be known and even if they are known, it might be non-trivial or downright impossible to aggregate the given rewards into a single one that can be fed to the maestro's learning procedure to achieve the desired solution concept, more on this in the next lesson.

### Independent learning
The opposite approach is for each agent to learn using an independent learning procedure, removing the need for a centralized authority, its dependency on the number of players and the issue of handling the rewards. This approach has proven to be much more successfull, but classical algorithm from RL might not be suited to the task.

#### Independent Q-learning
Consider the case in which each agents learns using the standard Q-learning algorithm. On each round, each player $i \in \{1, \dots, N\}$ observes a state $s \in \mathcal{S}$ and picks an action $a_i \in \mathcal{A}_i$, after each player picked their action $a_1, \dots, a_N$, the next state $s' = \tau(s, a)$ transitions according the joint action $a = (a_1, \dots, a_N)$ and each player receives reward $r_i$, which is used to update their respective value function $Q_i$ as
$$
	Q_i(s, a_i) =
		(1 - \alpha) Q_i(s, a_i)
		+ \alpha ( r_i + \gamma \max_{a'_i \in \mathcal{A}_i} Q_i(s', a'_i) )
$$
Because the transition from one state to another depends on the joint action $a = (a_1, \dots, a_N)$ and not on a single action, from a single player's point of view the transition function of the MDP depends on the other players actions, because the other players actively learn and are therefore non-stationary in their choice of action, the transition function is non-stationary. In general this means that independent learning does not converge, but in certains classes of games, it does.

### The goal of multi-agent reinforcement learning
In the classic RL case, a policy is optimal if it maximises the _return_, defined as the sum of the immediate reward obtained during play, in the multi-agent case, any player's return depends on the other players' actions and ultimately depends on their policies.

**If the return of a policy depends on the other policies, how do we define optimality?** There is no fixed answer to this question, in fact it depends on the underlying context and the objective of the learning procedure, for now we only focus on modeling the problem. To understand the different solutions concepts for MARL we will rely on game-theorical arguments.

In the meantime, we can evaluate the performance of a learning algorithm by its convergence rate to a specific solution concept, viz. we define a specific policy for each player to which we want our algorithm to converge to and we try to understand
1. If the learning algorithm converges to the proposed solution (which the algorithm does not know).
2. If true, how fast is the convergence?

For example, going back to the football example, before the start of the game we decide _roughly_ how we want the players to play by defining different roles, which translate to different behaviours. Afterward we pick a learning algorithm and try to understand if the players actually learn to play as we intended.

This might seem very accidental, how can we be sure that the algorithm will converge to these behaviours that we randomly picked? This can be achieved by choosing these behaviours wisely from very well studied solution concepts, which we will tackle in the next lesson.

### Welcome to the PettingZoo
The Farama foundation, who maintains the Gymnasium library, work also on a MARL-oriented library called [PettingZoo](https://pettingzoo.farama.org/). They share the same interface, which is very useful when, hypothetically, you have been following a course using Gym and now you need to abruptly switch library to cover MARL topics.

In [1]:
from IPython.display import clear_output
import matplotlib.pyplot as plt
import numpy as np

def visualize(render):
	clear_output(wait=True)
	plt.imshow(render)
	plt.axis("off")
	plt.show()

### Tic-Tac-Toe

In [ ]:
from pettingzoo.classic import tictactoe_v3

env = tictactoe_v3.env(render_mode = "rgb_array")
env.reset(seed = 42)

for agent in env.agent_iter():
	obs, reward, termination, truncation, info = env.last()
	if termination or truncation:
		action = None
	else:
		legal_moves = obs["action_mask"]
		action = env.action_space(agent).sample(legal_moves)

	visualize(env.render())
	env.step(action)

env.close()

## Simple Spread

We know consider the game of [Simple Spread](https://pettingzoo.farama.org/environments/mpe/simple_spread/), where $N$ agents want to cover $N$ landmarks and are penalized if they collide with eachother. The reward is the same for every agent and is the sum of the minimum distance from each landmark to the closest agent.

In [2]:
from pettingzoo.mpe import simple_spread_v3

env = simple_spread_v3.parallel_env(
	N = 5,
	max_cycles = 50,
	continuous_actions = False,
	render_mode = "rgb_array"
)

observations, _ = env.reset()

while env.agents:
	actions = {agent: env.action_space(agent).sample() for agent in env.agents}

	visualize(env.render())
	observations, rewards, terminations, truncations, _ = env.step(actions)
env.close()

REWRITE FOR CONTINUOUS STATES

### Multi-agent Q-learning
We now make slight adjustments to the Q-learning implementation you are familiar with to make it work with PettingZoo, the main change is that now the action space is tied to the agent, therefore when choosing a possible action, we need to invoke ```env.action_space(agent)``` instead of ```env.action_space```.

For sake of generality, we also define a pair of encoding/decoding functions which we can use to map the action/state space to types which Python can hash, making it possible to use it in normal dictionaries for tabular learning.

In [ ]:
# TO REMOVE

from agents.tabular import QLearning

def encode(xs: np.ndarray) -> tuple[int]:
	return tuple(xs.flatten())

def decode(xs: tuple[int], size = (1, )) -> np.ndarray:
	if len(xs) == 1: return xs[0]
	return np.array(xs).reshape(size)

class MultiAgentQLearning(QLearning):
	def greedy(self, state, agent):
		default_action = encode(self.mdp.action_space(agent).sample())
		action = max(self.Q[state], key=self.Q[state].get, default=default_action)
		return decode(action)

	def explore(self, state, agent):
		if np.random.uniform() < self.epsilon:
			return self.mdp.action_space(agent).sample()
		else:
			return self.greedy(state, agent)

In [ ]:
# TO REMOVE

from tqdm import trange

EPISODES = 500
MAX_ITERATIONS = 100
SAVE_EVERY = 1

env = simple_spread_v3.parallel_env(
	max_cycles = MAX_ITERATIONS,
	continuous_actions = False,
	render_mode = "rgb_array"
)

algorithms = {
	# "agent_0": MultiAgentQLearning(env),
	# "adversary_0": MultiAgentQLearning(env, final_epsilon=1), # random
	agent: MultiAgentQLearning(env)
	for agent in env.possible_agents
}

# Set the epsilon decay value
for agent in algorithms.values():
	agent.e_decay = agent.epsilon / (MAX_ITERATIONS / 2)

for episode in trange(EPISODES):
	observations, _ = env.reset()

	observations_encoded = {
		agent: encode(observations[agent])
		for agent in env.possible_agents
	}

	# Keep track of the average error for each agent
	episode_error = {agent: [] for agent in env.possible_agents}

	while env.agents:
		actions = {
			agent: algorithms[agent].explore(observations_encoded[agent], agent)
			for agent in env.agents
		}
		
		# visualize(env.render())
		new_observations, rewards, terminations, truncations, infos = env.step(actions)

		new_observations_encoded = {
			agent: encode(new_observations[agent])
			for agent in env.possible_agents
		}

		for agent, algorithm in algorithms.items():
			# Encode the states and action
			error = algorithm.update(
				state = observations_encoded[agent],
				action = encode(actions[agent]),
				reward = rewards[agent],
				terminated = terminations[agent],
				s_prime = new_observations_encoded[agent]
			)
			episode_error[agent].append(error)

		observations_encoded = new_observations_encoded

	if episode % SAVE_EVERY == 0:
		for agent, algorithm in algorithms.items():
			# algorithm.history.append(algorithm.Q.copy())
			algorithm.error.append(np.average(episode_error[agent]))

	for algorithm in algorithms.values():
		algorithm.decay_epsilon()
		
env.close()

## Discrete actions and continuous state

In [4]:
from gymbase.vfa import LinearQApproximator

def multiagent_egreedy_policy_q(env, approximator, state, epsilon, agent):
	if np.random.rand() < epsilon:
		return env.action_space(agent).sample()
	return np.argmax(approximator.predict(state, action) for action in range(env.action_space(agent).n))

Train a number of agents using our approximate Q-learning implementation for a fixed number of episodes.

In [ ]:
from tqdm import trange

EPISODES = 1000
MAX_ITERATIONS = 100

# Discount parameter
GAMMA = 0.8

env = simple_spread_v3.parallel_env(
	max_cycles = MAX_ITERATIONS,
	continuous_actions = False,
	render_mode = "rgb_array"
)
observations, _ = env.reset()

# Each agent is represented by an instance of the LinearQApproximator class,
# which handles the weights for the value approximation.
approximators = {
	agent: LinearQApproximator(
		feature_extractor = lambda state: state,
		num_features = observations[agent].size,
		num_actions = env.action_space(agent).n,
		alpha = 0.01 # Mixing parameter
	) for agent in env.possible_agents
}

# For simplicity in this example, this parameter is shared across learners,
# but in general it is learner-specific.
epsilon = 1
min_epsilon = 0.1
epsilon_decay = (epsilon - min_epsilon) / EPISODES

# Keep track of the mean error and its standard deviation across learning
mean_error = {agent: [] for agent in env.possible_agents}
std_error = {agent: [] for agent in env.possible_agents}

for episode in trange(EPISODES):
	observations, _ = env.reset()

	# Keep track of the average error for each agent
	episode_error = {agent: [] for agent in env.possible_agents}

	while env.agents:
		# Compute the action using an approximate epsilon-greedy policy
		actions = {
			agent: multiagent_egreedy_policy_q(env, approximators[agent], observations[agent], epsilon, agent)
			for agent in env.agents
		}
		
		# Advance the simulation according to the action profile
		next_observations, rewards, terminations, truncations, infos = env.step(actions)

		for agent, approximator in approximators.items():
			# Observations in general do not coincide with states, because different agents
			# might have different views of the environment. It is not the case here.
			state, next_state = observations[agent], next_observations[agent]
			reward = rewards[agent]

			next_best_action = np.argmax(approximator.predict(next_state, action)
				for action in range(env.action_space(agent).n))

			target = reward + (GAMMA * approximator.predict(next_state, next_best_action))

			# Log the current error
			error = target - approximator.predict(state, actions[agent])
			episode_error[agent].append(error)

			# Update the weights of the predictor
			approximator.update(state, actions[agent], target)

		observations = next_observations

	for agent, errors in episode_error.items():
		mean_error[agent].append(np.average(errors))
		std_error[agent].append(np.std(errors))

	epsilon -= epsilon_decay
		
env.close()

100%|██████████| 1000/1000 [02:02<00:00,  8.15it/s]


Now plot the average error incurred by the agents across the episodes.

In [ ]:
skip = 3

x = range(EPISODES)[::skip]
for agent in env.possible_agents:
	std = np.array(std_error[agent])[::skip]
	error = np.array(mean_error[agent])[::skip]

	plt.plot(x, error, label = agent)
	# plt.fill_between(x, (error - std), (error + std), alpha = .1)

plt.legend()
plt.show()